[IPython Notebook](building_a_system.ipynb) |  [Python Script](building_a_system.py)

Building a System
============================

Setup
-----------------------------

From now on, we'll just quickly do common setup at the beginning of each tutorial.
For full gory details on the general concepts here, make sure to read [General Concepts](general_concepts).

We'll always start by doing our basic imports, setting up a logger, and initializing
a default bundle.

In [1]:
import phoebe2
from phoebe2 import u # units
import numpy as np
import matplotlib.pyplot as plt

logger = phoebe2.logger(clevel='INFO')

b = phoebe2.Bundle()

/usr/local/lib/python2.7/dist-packages/IPython/kernel/__init__.py:13: ShimWarning: The `IPython.kernel` package has been deprecated. You should import from ipykernel or jupyter_client instead.
  "You should import from ipykernel or jupyter_client instead.", ShimWarning)


Default Systems
------------------------

Although the default empty binary doesn't include a system, there are available
constructors that create default systems.  To create a simple binary with component tags
'binary', 'primary', and 'secondary' (as above), you could call:

In [2]:
b = phoebe2.Bundle.default_binary()

/usr/local/lib/python2.7/dist-packages/astropy/units/quantity.py:732: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  return super(Quantity, self).__eq__(other)


In [3]:
print b.hierarchy


    orbit:binary 
    
        star:primary 
        star:secondary
    



To build the same binary but as an overcontact, you would call:

In [4]:
b = phoebe2.Bundle.default_binary(overcontact=True)

In [5]:
print b.hierarchy


    orbit:binary 
    
        star:primary 
        star:secondary 
        envelope:common_envelope
    



For more details on dealing with overcontact systems, see the [Overcontact Example Script](../examples/minimal_overcontact)

To build a hierarchical triple system, you would call:

In [6]:
b = phoebe2.Bundle.default_triple()

In [7]:
print b.hierarchy


    orbit:outer 
    
        orbit:inner 
        
            star:starA 
            star:starB
         
        star:starC
    



Note that by default this set's the inner-orbit as the primary component in the outer-orbit.  To override this behavior, call:

In [8]:
b = phoebe2.Bundle.default_triple(inner_as_primary=False)

In [28]:
print b.hierarchy


    orbit:binary 
    
        star:primary 
        star:secondary
    



You can also choose to make the inner-binary in a triple system an overcontact:

In [29]:
b = phoebe2.Bundle.default_triple(inner_as_overcontact=True)

In [30]:
print b.hierarchy


    orbit:outer 
    
        orbit:inner 
        
            star:starA 
            star:starB 
            envelope:common_envelope
         
        star:starC
    



For more details on dealing with triple systems, see the [Hierarchical Triple Example Script](../examples/hierarchical_triple)

Adding Components Manually
--------------------

By default, an empty bundle does not contain any information about our system.

So, let's first start by adding a few stars.  Here we'll call the generic add_component method.  This method works for any type of component in the system - stars, orbits, planets, disks, rings, spots, etc.  The first argument needs to be a callable or the name of a callable in phoebe2.parameters.component which include the following options:

* orbit
* star
* envelope

add_component also takes a keyword argument for the 'component' tag.  Here we'll give them component tags 'primary' and 'secondary' - but note that these are merely convenience labels and do not hold any special roles.  Some tags, however, are forbidden if they clash with other tags or reserved values - so if you get error stating the component tag is forbidden, try using a different string.

In [10]:
b.add_component(phoebe2.component.star, component='primary')
b.add_component('star', component='secondary')

<ParameterSet: 13 parameters | qualifiers: gravblaw, teff, abun, pot, ld_func_bol, period, ld_coeffs_bol, rpole, alb_bol, syncpar, gravb_bol, freq, incl>

But there are also shortcut methods for add_star and add_orbit.  In these cases you don't need to provide the function, but only the component tag of your star/orbit.

Any of these functions also accept values for any of the qualifiers of the created parameters.

In [11]:
b.add_star('extrastarforfun', teff=6000)

<ParameterSet: 13 parameters | qualifiers: gravblaw, teff, abun, pot, ld_func_bol, period, ld_coeffs_bol, rpole, alb_bol, syncpar, gravb_bol, freq, incl>

Here we call the add_component method of the bundle with several arguments:

* a function (or the name of a function) in phoebe2.parameters.component.  This 
  function tells the bundle what parameters need to be added.
* component: the tag that we want to give this component for future reference.
* any additional keyword arguments: you can also provide initial values for Parameters 
  that you know will be created.  In the last example you can see that the
  effective temperature will already be set to 6000 (in default units which is K).

and then we'll do the same to add an orbit:

In [12]:
b.add_orbit('binary')

<ParameterSet: 14 parameters | qualifiers: dpdt, ecc, dperdt, per0, period, long_an, q, t0_supconj, phshift, t0_perpass, freq, incl, deccdt, sma>

Defining the Hierarchy
---------------------------------

At this point all we've done is add a bunch of Parameters to our Bundle, but 
we still need to specify the hierarchical setup of our system.

Here we want to place our two stars (with component tags 'primary' and 'secondary') in our
orbit (with component tag 'binary').  This can be done with several different syntaxes:

In [13]:
b.set_hierarchy(phoebe2.hierarchy.binaryorbit, b['binary'], b['primary'], b['secondary'])

or

In [14]:
b.set_hierarchy(phoebe2.hierarchy.binaryorbit(b['binary'], b['primary'], b['secondary']))

If you access the value that this set, you'll see that it really just resulted
in a simple string representation:

In [15]:
b.get_hierarchy()

<HierarchyParameter: orbit:binary(star:primary, star:secondary)>

We could just as easily have used this string to set the hierarchy:

In [16]:
b.set_hierarchy('orbit:binary(star:primary, star:secondary)')

If at any point we want to flip the primary and secondary components or make
this binary a triple, its seriously as easy as changing this hierarchy and
everything else will adjust as needed (including cross-ParameterSet constraints, 
and datasets)

The Hierarchy Parameter
-----------------------------

Setting the hierarchy just sets the value of a single parameter (although it may take some time because it also does a lot of paperwork and manages constraints between components in the system).  You can access that parameter as usual:

In [17]:
b['hierarchy@system']

<HierarchyParameter: orbit:binary(star:primary, star:secondary)>

or through any of these shortcuts:

In [18]:
b.get_hierarchy()

<HierarchyParameter: orbit:binary(star:primary, star:secondary)>

In [19]:
b.hierarchy

<HierarchyParameter: orbit:binary(star:primary, star:secondary)>

This hierarchy parameter then has several methods unique to itself.  You can, for instance, list the component tags of all the stars or orbits in the hierarchy:

In [20]:
print b.hierarchy.get_stars()

['primary', 'secondary']


In [21]:
print b.hierarchy.get_orbits()

['binary']


Or you can ask for the component tag of the top-level item in the hierarchy

In [22]:
print b.hierarchy.get_top()

binary


And request the parent, children, child, or sibling of any item in the hierarchy

In [23]:
print b.hierarchy.get_parent_of('primary')

binary


In [24]:
print b.hierarchy.get_children_of('binary')

['primary', 'secondary']


In [25]:
print b.hierarchy.get_child_of('binary', 0)  # here 0 means primary component, 1 means secondary

primary


In [26]:
print b.hierarchy.get_sibling_of('primary')

secondary


We can also check whether a given component (by component tag) is the primary or secondary component in its parent orbit.  Note that here its just a coincidence (although on purpose) that the component tag is also 'secondary'.

In [27]:
print b.hierarchy.get_primary_or_secondary('secondary')

secondary
